In [ ]:
from google.colab import drive
from lxml import etree as et
import xml.etree.ElementTree as etree
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
from sklearn.metrics import f1_score, classification_report

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def feature_table(xml_path, out_path):
    df = pd.DataFrame(columns=['id', 'word_lower', 'capital_letter', 'subpart_of_speech', 'pause_type', 'end_of_sentence'])

    tree = et.parse(xml_path)
    root = tree.getroot()
    words = []
    eos = []
    pause_type = []
    subpart_of_speech = []
    cap_let = []

    for sentence in root.iter('sentence'):
        for word in sentence.iter('word'):
            if 'original' in word.attrib.keys():
                words.append(word.attrib['original'].lower())
                if word.attrib['original'][0].isupper():
                    cap_let.append(1)
                else:
                    cap_let.append(0)
            else:
                words.append('none')
                cap_let.append(0)

        for content in sentence.iter('content'):
            if 'PunktEnd' in content.attrib.keys():
                eos.append(1)
            else:
                eos.append(0)

            if 'PauseType' in content.attrib.keys():
                pause_type.append(content.attrib['PauseType'])
            else:
                pause_type.append(0)
        for dictitem in sentence.iter('dictitem'):
            subpart_of_speech.append(int(dictitem.attrib['subpart_of_speech']))

    for i in range(len(pause_type)):
        df.loc[df.shape[0]] = [i, words[i], cap_let[i], subpart_of_speech[i], pause_type[i], eos[i]]
    df.to_csv(out_path, index=False)


In [ ]:
feature_table('/content/drive/My Drive/Colab Notebooks/JackLondon.MartinEden.Result.xml','/content/drive/My Drive/Colab Notebooks/data1.csv')

Логистическая регрессия

In [ ]:
def model(table_path):
    test_df = pd.read_csv(table_path)

    features = np.asarray(test_df.iloc[0:test_df.shape[0], 2:5])
    labels = test_df.iloc[0:test_df.shape[0], 5]

    x_train, x_valid, y_train, y_valid = train_test_split(features, labels, stratify=labels)
    model = LogisticRegression(random_state = 0).fit(x_train, y_train)
    y_pred = model.predict(x_valid)


    print(f1_score(labels[0:len(y_pred)], y_pred, average='micro'))
    print(classification_report(labels[0:len(y_pred)], y_pred))

In [ ]:
model('/content/drive/My Drive/Colab Notebooks/data1.csv')

0.6622639017302866
              precision    recall  f1-score   support

           0       0.78      0.79      0.79      5958
           1       0.20      0.19      0.19      1613

    accuracy                           0.66      7571
   macro avg       0.49      0.49      0.49      7571
weighted avg       0.66      0.66      0.66      7571



Гауссовский наивный байесовский

In [ ]:
def modelGaussian(table_path):
    test_df = pd.read_csv(table_path)

    features = np.asarray(test_df.iloc[0:test_df.shape[0], 2:5])
    labels = test_df.iloc[0:test_df.shape[0], 5]

    x_train, x_valid, y_train, y_valid = train_test_split(features, labels, stratify=labels)
    model = GaussianNB().fit(x_train, y_train)
    y_pred = model.predict(x_valid)

    print(f1_score(labels[0:len(y_pred)], y_pred, average='micro'))
    print(classification_report(labels[0:len(y_pred)], y_pred))

In [ ]:
modelGaussian('/content/drive/My Drive/Colab Notebooks/data1.csv')

0.6724342887333246
              precision    recall  f1-score   support

           0       0.79      0.80      0.79      5958
           1       0.22      0.21      0.22      1613

    accuracy                           0.67      7571
   macro avg       0.51      0.51      0.51      7571
weighted avg       0.67      0.67      0.67      7571

